In [2]:
# # Install fixes to crewai and crewai_tools:
# ! pip uninstall -y crewai crewai_tools
# ! pip install git+https://github.com/piotrm0/crewAI@piotrm/wraps
# ! pip install git+https://github.com/piotrm0/crewAI-tools@piotrm/base_tool 
# ! pip list | grep crew

In [ ]:
from pathlib import Path
import sys
import os

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

# If you have these in your env, they seem to be picked up by litellm for
# requests which should not use them and thus cause problems.
del os.environ['AWS_ACCESS_KEY_ID']
del os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
import trulens.core.utils.threading # to patch Thread and ThreadPoolExecutor
from trulens.core.session import TruSession
from trulens.apps.custom import TruCustomApp

session = TruSession()
session.experimental_enable_feature("otel_tracing")

session.reset_database()
session.start_dashboard(force=True, port=8080)

In [5]:
from evaluator_crew.eval_crew import EvaluatorCrew
from marketing_crew.marketing_crew import MarketingPostsCrew

e2e_evaluator_crew = EvaluatorCrew()
e2e_evaluator_crew.crew = e2e_evaluator_crew.crew()

marketing_posts_crew = MarketingPostsCrew()
marketing_posts_crew.crew = marketing_posts_crew.crew()

tru_marketing_posts_crew = TruCustomApp(marketing_posts_crew, app_name="MarketingPostsCrew", app_version="1.0.0", feedbacks=[])

In [7]:
os.environ["AZURE_API_BASE"] = "https://sfc-apim-sweden.azure-api.net"

In [8]:
inputs = {
      'customer_domain': 'crewai.com',
            'project_description': """
            CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients.
            This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions,
            emphasizing ease of use, scalability, and integration capabilities.
            The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

            Customer Domain: AI and Automation Solutions
            Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise clients.
            Secondary goal: During the execution of the multi-agent system/tasks, evaluate the quality of answers at each step.
            """,
            'previous_marketing_post': "",
            'feedback': "",
}

with tru_marketing_posts_crew as recorder:
    result = marketing_posts_crew.crew.kickoff(inputs=inputs)

# Agent: Lead Market Analyst
## Task: Conduct a thorough research about the customer and competitors in the context of crewai.com. Make sure you find any interesting and relevant information given the current year is 2024. We are working with them on the following project: 
            CrewAI, a leading provider of multi-agent systems, aims to revolutionize marketing automation for its enterprise clients.
            This project involves developing an innovative marketing strategy to showcase CrewAI's advanced AI-driven solutions,
            emphasizing ease of use, scalability, and integration capabilities.
            The campaign will target tech-savvy decision-makers in medium to large enterprises, highlighting success stories and the transformative potential of CrewAI's platform.

            Customer Domain: AI and Automation Solutions
            Project Overview: Creating a comprehensive marketing campaign to boost awareness and adoption of CrewAI's services among enterprise 

In [ ]:
for record in recorder.records:
    print(record.record_id)
    print(record)